<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Vizzu - Create Animated Pie Chart

**Tags:** #vizzu #animation #piechart #data #visualization #python

**Author:** [Alexandre Petit](https://www.linkedin.com/in/alexandre-petit-24a87a219/)

**Description:** This notebook will show how to create an animated pie chart with Vizzu. It is usefull for organizations to visualize data in an interactive way.

**References:**
- [Vizzu Documentation](https://vizzu.io/docs/getting-started/introduction)
- [Vizzu Tutorials](https://vizzu.io/tutorials)

## Input

### Import libraries

In [ ]:
import vizzu

### Setup Variables
- `data`: dataset to be used for the pie chart

In [ ]:
data = [
    {"name": "A", "value": 10},
    {"name": "B", "value": 20},
    {"name": "C", "value": 30},
    {"name": "D", "value": 40},
]

## Model

### Create Animated Pie Chart

This function will create an animated pie chart with Vizzu.

In [ ]:
vizzu.pie_chart(data, animate=True)

## Output

### Display result

In [ ]:
vizzu.show()